In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import numpy as np
import konlpy

In [2]:
import requests
import pandas as pd
import urllib.request as req
from bs4 import BeautifulSoup

In [3]:
news_df = pd.DataFrame(columns = ['news_text','category'])
news_df

,news_text,category


In [4]:
import requests
import pandas as pd
import urllib.request as req
from bs4 import BeautifulSoup
cat=[100,102,103,105,104]
page=0
def make_urllist(cat,page):
    urllist=[]
    for i in range(cat):
        url = 'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1='+str(cat)+'&date=%2000:00:00&page='+str(page)
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36'}
        news = requests.get(url, headers=headers)
        soup = BeautifulSoup(news.text,'html.parser')
        li = []
        text = soup.find_all(class_='cluster_item as_line')
        for i in text:
            d ={}
            d['title']=i.find(class_='cluster_text').get_text()
            d['cat']=cat
            d['title'] = re.sub('\n','',d['title'])
            d['title'] = re.sub('\t','',d['title'])
            li.append(d)
            df=pd.DataFrame(li)
            page+=1
        return df

In [5]:
news_df = pd.DataFrame(columns = ['news_text','category'])
news_df

,news_text,category


In [6]:
cats=[100,102,103,105,104]
ind = 0
for cat in cats:   
    for i in range(70):
        
        url = 'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1='+str(cat)+'&date=%2000:00:00&page='+str(i+1)
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36'}
        news = requests.get(url, headers=headers)
        soup = BeautifulSoup(news.text,'html.parser')
        links = soup.select(' #main_content > div > div > div > div > div > ul > li > div > a')
        # 첫번째 뉴스,두번쨰 세번째 넘버링 하기 위해 enumerate사용
        #주소를 가져온 뒤 거기서 id로 준 값에 해당하는 내용 추출
        newslinks=[]
        for j, a in enumerate(links):            
            newslink = a.get('href')
            newslinks.append(newslink)
            newslinks = list(set(newslinks))        
        
        for link in newslinks:
            res2 = requests.get(link,headers = headers).text
            soup2 = BeautifulSoup(res2,'html.parser')
            news = soup2.find('div',id = "articleBodyContents").text
            news = re.sub('\n','',news)
            news = re.sub('\t','',news)
            news_df.loc[ind] = [news,cat]
            ind += 1
news_df

,news_text,category
0,“곧 저에 대한 보복 수사 준비할 듯” 조롱김용민 더불어민주당 의원. [연합][헤럴...,100
1,송영길 더불어민주당 전 대표가 20일 서울시 여의도 이룸센터 앞 전국장애인차별철폐연...,100
2,"김용민 의원, 광주고검장 문자 그대로 실명 공개김용민 더불어민주당 의원 페이스북 캡...",100
3,한국투자증권 보고서[이데일리 이지현 기자] 이창용 한국은행 총재가 오늘부터 4년의 ...,100
4,"[[the300]]윤호중, 박지현 더불어민주당 공동비상대책위원장이 20일 오전 서울...",100
...,...,...
10049,채굴업체 비트리버 등 11곳 대상“인권유린” 러 635명 비자 제한조 바이든 미국 ...,104
10050,미국 연방정부가 대중교통 마스크 착용 의무화 조치를 무효화한 법원 결정에 항소하겠다...,104
10051,마스크 착용 요구 안내. 〈사진-로이터·연합뉴스〉 미국 정부가 대중교...,104
10052,중도보수 성향의 에마뉘엘 마크롱 대통령과 극우 성향의 마린 르펜 국민연합 후보가 2...,104


In [7]:
news_df.to_csv('news_dff.csv',mode='w')

In [8]:
f_df = pd.read_csv('news_dff.csv',index_col=0)
f_df.shape

(10054, 2)

In [9]:
f_df.head()

,news_text,category
0,“곧 저에 대한 보복 수사 준비할 듯” 조롱김용민 더불어민주당 의원. [연합][헤럴...,100
1,송영길 더불어민주당 전 대표가 20일 서울시 여의도 이룸센터 앞 전국장애인차별철폐연...,100
2,"김용민 의원, 광주고검장 문자 그대로 실명 공개김용민 더불어민주당 의원 페이스북 캡...",100
3,한국투자증권 보고서[이데일리 이지현 기자] 이창용 한국은행 총재가 오늘부터 4년의 ...,100
4,"[[the300]]윤호중, 박지현 더불어민주당 공동비상대책위원장이 20일 오전 서울...",100


In [10]:
f_df.category.value_counts()

100    2800
102    2752
104    1982
105    1400
103    1120
Name: category, dtype: int64

In [11]:
f_df_100 = f_df[f_df.category==100][:1000]
f_df_102 = f_df[f_df.category==102][:1000]
f_df_103 = f_df[f_df.category==103][:1000]
f_df_104 = f_df[f_df.category==104][:1000]
f_df_105 = f_df[f_df.category==105][:1000]

In [12]:
import pandas as pd
news1_df = pd.concat([f_df_100,f_df_102,f_df_103,f_df_104,f_df_105],axis=0)
news1_df.shape

(5000, 2)

In [13]:
news1_df.head()

,news_text,category
0,“곧 저에 대한 보복 수사 준비할 듯” 조롱김용민 더불어민주당 의원. [연합][헤럴...,100
1,송영길 더불어민주당 전 대표가 20일 서울시 여의도 이룸센터 앞 전국장애인차별철폐연...,100
2,"김용민 의원, 광주고검장 문자 그대로 실명 공개김용민 더불어민주당 의원 페이스북 캡...",100
3,한국투자증권 보고서[이데일리 이지현 기자] 이창용 한국은행 총재가 오늘부터 4년의 ...,100
4,"[[the300]]윤호중, 박지현 더불어민주당 공동비상대책위원장이 20일 오전 서울...",100


In [14]:
news1_df.to_csv('news1_df_1000.csv',index=False)

In [15]:
news2_df = pd.read_csv('news1_df_1000.csv')
news2_df.head()

,news_text,category
0,“곧 저에 대한 보복 수사 준비할 듯” 조롱김용민 더불어민주당 의원. [연합][헤럴...,100
1,송영길 더불어민주당 전 대표가 20일 서울시 여의도 이룸센터 앞 전국장애인차별철폐연...,100
2,"김용민 의원, 광주고검장 문자 그대로 실명 공개김용민 더불어민주당 의원 페이스북 캡...",100
3,한국투자증권 보고서[이데일리 이지현 기자] 이창용 한국은행 총재가 오늘부터 4년의 ...,100
4,"[[the300]]윤호중, 박지현 더불어민주당 공동비상대책위원장이 20일 오전 서울...",100
